# Bibliotecas

In [1]:
from metapy_toolbox import *
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split


from fundacao import *

# Carregando dados

In [2]:
# df = pd.read_excel(r"/home/wmpjrufg/Documents/fundaIA/assets/toy_problem.xlsx") # Prof. Wanderlei
# df = pd.read_excel(r"C:\git-projetos\fundaIA\assets\toy_problem_copy_2.xlsx") # Prof. Wanderlei casa
df = pd.read_excel(r"C:\git-projetos\fundaIA\assets\toy_problem_copy_3.xlsx") # Prof. Wanderlei casa
n_comb = 3
f_ck = 25000
cob_m = 0.04
df

,Elemento,ap (m),bp (m),spt,solo,xg (m),yg (m),Fz-c1,Mx-c1,My-c1,Fz-c2,Mx-c2,My-c2,Fz-c3,Mx-c3,My-c3
0,P16,0.25,1.2,43,argila,27.75,18.065,377.3,-1.9,4.4,259.2,-27.8,0.1,383.8,27.7,0.2


# Pop inicial de possíveis soluções

In [3]:
n_population = 1000
seed = 42

In [4]:
x_lower = [0.60] * len(df) * 3
x_upper = [5.00] * len(df) * 3
n_dimensions = len(df) * 3
pop2d = initial_population_01(n_population, n_dimensions, x_lower, x_upper, seed, use_lhs=True)

# Gerando dataset completo

In [5]:
cols = []
for i in range(len(df)):
    cols.append(f'hx{i}')
    cols.append(f'hy{i}')
    cols.append(f'hz{i}')
df_pop = pd.DataFrame(pop2d, columns=cols)
args = [df, n_comb, f_ck, cob_m]
def wrapper(x):
    return obj_teste(x, args)[0]
vol = Parallel(n_jobs=-1)(delayed(wrapper)(x) for x in pop2d)
df_pop['volume (m3)'] = vol
df_pop

,hx0,hy0,hz0,volume (m3)
0,3.694195,0.835669,3.187822,470285.728814
1,1.032532,1.250786,0.705707,124346.228346
2,0.997051,3.896541,1.884236,7.320353
3,2.578018,1.377168,3.548322,19038.857263
4,3.069967,1.485180,4.646049,21.183442
...,...,...,...,...
995,2.807738,1.000133,1.046385,333225.531376
996,1.386895,4.671714,2.532520,16.408641
997,1.955984,3.942638,0.601985,4.642346
998,3.074723,4.478719,1.661917,22.885967


In [6]:
oi = df_pop.sort_values(by="volume (m3)", ascending=True)
oi

,hx0,hy0,hz0,volume (m3)
715,0.683911,2.377185,0.710025,1.154347e+00
234,0.982116,2.364953,0.620754,1.441799e+00
652,1.253179,2.399256,0.609461,1.832464e+00
838,1.553776,1.784319,0.696158,1.930051e+00
929,0.707628,1.863086,1.746704,2.302805e+00
...,...,...,...,...
430,0.905968,0.680701,3.589561,1.063909e+06
275,0.797920,0.764579,3.991382,1.074031e+06
92,0.669001,0.857380,0.904150,1.205538e+06
623,0.776736,0.739466,4.698865,1.206813e+06


# Aprendizado de máquina do dataset

### Separação em x e y

In [7]:
x = df_pop.drop(columns=['volume (m3)'])
y = df_pop[['volume (m3)']].reset_index(drop=True)
y

,volume (m3)
0,470285.728814
1,124346.228346
2,7.320353
3,19038.857263
4,21.183442
...,...
995,333225.531376
996,16.408641
997,4.642346
998,22.885967


In [8]:
x_treino, x_teste, y_treino, y_teste= train_test_split(x, y, test_size=0.30, random_state=seed)
df_gpr = aprendizado_maquina(x_treino, y_treino, x_teste, y_teste)
df_gpr

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.2min finished


,modelo,R2 Treino,R2 Teste,MAE,RMSE,arquivo
0,gpr_k02,1.0,0.912990,21340.851413,52168.497674,modelos\gpr_k02_pop_700.pkl
1,gpr_k01,1.0,0.884757,24072.522897,60038.534947,modelos\gpr_k01_pop_700.pkl
2,gpr_k03,1.0,0.884757,24072.521507,60038.539014,modelos\gpr_k03_pop_700.pkl


In [9]:
x_treino, x_teste, y_treino, y_teste= train_test_split(x, y, test_size=0.25, random_state=seed)
df_gpr = aprendizado_maquina(x_treino, y_treino, x_teste, y_teste)
df_gpr

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.2min finished


,modelo,R2 Treino,R2 Teste,MAE,RMSE,arquivo
0,gpr_k02,1.0,0.911075,21623.978491,52477.749541,modelos\gpr_k02_pop_750.pkl
1,gpr_k03,1.0,0.880053,25139.458025,60947.633608,modelos\gpr_k03_pop_750.pkl
2,gpr_k01,1.0,0.880053,25139.456507,60947.637391,modelos\gpr_k01_pop_750.pkl


In [10]:
x_treino, x_teste, y_treino, y_teste= train_test_split(x, y, test_size=0.20, random_state=seed)
df_gpr = aprendizado_maquina(x_treino, y_treino, x_teste, y_teste)
df_gpr

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  1.6min finished


,modelo,R2 Treino,R2 Teste,MAE,RMSE,arquivo
0,gpr_k02,1.0,0.965388,14968.051208,30287.49203,modelos\gpr_k02_pop_800.pkl
1,gpr_k01,1.0,0.957184,16772.764578,33686.22418,modelos\gpr_k01_pop_800.pkl
2,gpr_k03,1.0,0.957184,16772.766465,33686.22920,modelos\gpr_k03_pop_800.pkl
